In [286]:
import pandas as pd
import numpy as np
train = pd.read_excel('train.xlsx')
test = pd.read_excel('test.xlsx')

In [287]:
train

,Unnamed: 0,월,일,역번호,역명,승하차,05시-06시,06시-07시,07시-08시,08시-09시,...,지점_y,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 풍속(m/s),최대 순간 풍속 풍향(deg)
0,0,1,1,1150,설화명곡,승차,26,39,63,105,...,991,옥포,2023-01-01,-0.9,-8.4,7.8,0.0,8.8,1.8,335.1
1,1,1,1,1150,설화명곡,하차,0,65,49,55,...,991,옥포,2023-01-01,-0.9,-8.4,7.8,0.0,8.8,1.8,335.1
2,2,1,1,1160,화원,승차,19,44,40,115,...,991,옥포,2023-01-01,-0.9,-8.4,7.8,0.0,8.8,1.8,335.1
3,3,1,1,1160,화원,하차,4,57,44,76,...,991,옥포,2023-01-01,-0.9,-8.4,7.8,0.0,8.8,1.8,335.1
4,4,1,1,1170,대곡,승차,26,48,66,117,...,991,옥포,2023-01-01,-0.9,-8.4,7.8,0.0,8.8,1.8,335.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133037,133037,12,31,3390,지산,하차,3,18,28,151,...,860,신암,2024-12-31,4.2,0.7,7.2,0.0,10.8,1.4,293.0
133038,133038,12,31,3400,범물,승차,96,97,289,381,...,860,신암,2024-12-31,4.2,0.7,7.2,0.0,10.8,1.4,293.0
133039,133039,12,31,3400,범물,하차,47,30,92,228,...,860,신암,2024-12-31,4.2,0.7,7.2,0.0,10.8,1.4,293.0
133040,133040,12,31,3410,용지,승차,27,46,138,240,...,860,신암,2024-12-31,4.2,0.7,7.2,0.0,10.8,1.4,293.0


In [288]:
import re

def compute_usage(DF: pd.DataFrame) -> pd.DataFrame:
    """
    주어진 DataFrame에서 시간대별 및 지정된 불필요 컬럼을 제거하고,
    ['일자', '역명'] 기준으로 '승하차'별 '일계' 합계를 계산해 '이용량'을 생성합니다.

    Args:
        DF (pd.DataFrame): 원본 DataFrame

    Returns:
        pd.DataFrame: ['일자', '역명']별 '이용량' 및 상수 컬럼이 결합된 최종 DataFrame
    """
    # 1) 시간대별 칼럼 자동 수집
    time_cols = [
        '05시-06시','06시-07시','07시-08시','08시-09시','09시-10시',
        '10시-11시','11시-12시','12시-13시','13시-14시','14시-15시',
        '15시-16시','16시-17시','17시-18시','18시-19시','19시-20시',
        '20시-21시','21시-22시','22시-23시','23시-24시'
    ]

    # 2) 중간 처리: 시간대별 및 기타 불필요 컬럼 제거
    drop_list = time_cols + [
        'Unnamed: 0','PM10등급','PM2.5등급','역번호','일시',
        'nearest_weather_station','동','latitude','longitude',
        '지점_x','지점_y','지점명'
    ]
    df2 = DF.drop(columns=[c for c in drop_list if c in DF.columns])

    # 3) 그룹 키 정의
    group_keys = ['일자', '역명','날짜']

    # 4) 그룹별로 나머지 컬럼이 모두 동일한지 확인할 컬럼 목록
    const_cols = [c for c in df2.columns
                  if c not in group_keys + ['승하차', '일계']]

    # 5) 일관성 검사: nunique()가 1 초과인 값이 있으면 경고 출력
    nuniques = df2.groupby(group_keys)[const_cols].nunique()
    inconsistent = nuniques[nuniques > 1].stack()
    if not inconsistent.empty:
        print("⚠️ 다음 컬럼들이 그룹별로 값이 다릅니다:")
        print(inconsistent)
    else:
        print("✅ 모든 상수 컬럼 값이 그룹별로 동일합니다.")

    # 6) ‘이용량’ 계산: 승차·하차 ‘일계’ 합계
    df_usage = (
        df2
        .groupby(group_keys)['일계']
        .sum()
        .reset_index(name='이용량')
    )

    # 7) 상수 정보(날씨, 지점명 등)와 병합
    df_const = (
    df2
    .groupby(group_keys)[const_cols]
    .first()
    .reset_index()
    )
    df_final = pd.merge(df_usage, df_const, on=group_keys, how='left')

    # 결과 반환
    return df_final



In [289]:
traindf = compute_usage(train)
testdf = compute_usage(test)

✅ 모든 상수 컬럼 값이 그룹별로 동일합니다.
✅ 모든 상수 컬럼 값이 그룹별로 동일합니다.


In [ ]:
traindf['이용량'] = (
    traindf
    .groupby('역명')['이용량']
    .transform(lambda x: x.where(x != 0, x.mean()))
)

In [291]:
traindf[traindf['이용량']==0]

,일자,역명,날짜,이용량,월,일,년,요일,PM10수치,PM2.5수치,호선,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 풍속(m/s),최대 순간 풍속 풍향(deg)


In [292]:
traindf.shape

(66521, 18)

In [293]:
# 1) (일자, 역명)별 그룹 크기 계산
group_sizes = train.groupby(['일자','역명']).size()

# 2) 분포 출력: 몇 개 그룹이 1, 2, 3… 개의 행을 가지는지
print(group_sizes.value_counts())

# 3) 2가 아닌 그룹만 보기
bad = group_sizes[group_sizes != 2]
if bad.empty:
    print("✅ 모든 (일자, 역명) 그룹에 정확히 2개의 행이 있습니다.")
else:
    print("⚠️ 행 개수가 2가 아닌 그룹이 있습니다:")
    print(bad.head(10))

2    66521
Name: count, dtype: int64
✅ 모든 (일자, 역명) 그룹에 정확히 2개의 행이 있습니다.


In [294]:
# 숫자형 컬럼만 골라서
num_cols = traindf.select_dtypes(include='number').columns

for col in num_cols:
    n_missing = traindf[col].isna().sum()
    if n_missing > 0:
        median_val = traindf[col].median()
        # inplace 대신 대입문 사용
        traindf[col] = traindf[col].fillna(median_val)
        print(f"✅ '{col}'의 결측 {n_missing}개를 평균 {median_val:.2f}로 채웠습니다.")

✅ 'PM10수치'의 결측 8475개를 평균 26.00로 채웠습니다.
✅ 'PM2.5수치'의 결측 8443개를 평균 14.00로 채웠습니다.
✅ '평균기온(°C)'의 결측 13개를 평균 16.40로 채웠습니다.
✅ '최저기온(°C)'의 결측 13개를 평균 11.00로 채웠습니다.
✅ '최고기온(°C)'의 결측 13개를 평균 22.40로 채웠습니다.
✅ '일강수량(mm)'의 결측 3개를 평균 0.00로 채웠습니다.
✅ '최대 순간 풍속(m/s)'의 결측 21개를 평균 6.60로 채웠습니다.
✅ '평균 풍속(m/s)'의 결측 211개를 평균 1.30로 채웠습니다.
✅ '최대 순간 풍속 풍향(deg)'의 결측 21개를 평균 171.30로 채웠습니다.


In [295]:
# 숫자형 컬럼만 골라서
num_cols = testdf.select_dtypes(include='number').columns

for col in num_cols:
    n_missing = testdf[col].isna().sum()
    if n_missing > 0:
        median_val = testdf[col].median()
        # inplace 대신 대입문 사용
        testdf[col] = testdf[col].fillna(median_val)
        print(f"✅ '{col}'의 결측 {n_missing}개를 평균 {median_val:.2f}로 채웠습니다.")

✅ 'PM10수치'의 결측 566개를 평균 33.00로 채웠습니다.
✅ 'PM2.5수치'의 결측 558개를 평균 17.00로 채웠습니다.


In [296]:
traindf

,일자,역명,날짜,이용량,월,일,년,요일,PM10수치,PM2.5수치,호선,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 풍속(m/s),최대 순간 풍속 풍향(deg)
0,2023-01-01,각산,2023-01-01,4298.0,1,1,2023,일,26.0,14.0,1,2.3,-2.3,7.1,0.0,9.5,1.1,310.8
1,2023-01-01,감삼,2023-01-01,5624.0,1,1,2023,일,41.0,34.0,2,1.5,-4.1,7.3,0.0,10.2,2.2,17.7
2,2023-01-01,강창,2023-01-01,4337.0,1,1,2023,일,40.0,31.0,2,1.5,-4.1,7.3,0.0,10.2,2.2,17.7
3,2023-01-01,건들바위,2023-01-01,1550.0,1,1,2023,일,45.0,34.0,3,2.3,-2.3,7.1,0.0,9.5,1.1,310.8
4,2023-01-01,경대병원,2023-01-01,5150.0,1,1,2023,일,42.0,31.0,2,2.3,-2.3,7.1,0.0,9.5,1.1,310.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66516,2024-12-31,학정,2024-12-31,710.0,12,31,2024,화,39.0,17.0,3,3.1,-3.5,7.7,0.0,12.7,1.9,274.7
66517,2024-12-31,해안,2024-12-31,6099.0,12,31,2024,화,26.0,14.0,1,4.2,0.7,7.2,0.0,10.8,1.4,293.0
66518,2024-12-31,현충로,2024-12-31,7165.0,12,31,2024,화,38.0,14.0,1,3.8,-0.4,7.1,0.0,9.9,1.7,237.9
66519,2024-12-31,화원,2024-12-31,9447.0,12,31,2024,화,30.0,13.0,1,2.2,-5.3,7.2,0.0,12.9,2.6,323.1


In [297]:
traindf[traindf['이용량']==0]

,일자,역명,날짜,이용량,월,일,년,요일,PM10수치,PM2.5수치,호선,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 풍속(m/s),최대 순간 풍속 풍향(deg)


In [298]:
#날짜는 lightgbm 적용 x => drop
traindf.drop(columns= ['일자','날짜'], inplace= True)

from sklearn.preprocessing import LabelEncoder

# 1) 역명 인코딩
le_station = LabelEncoder()
traindf['역명_le'] = le_station.fit_transform(traindf['역명'])

# 2) 요일 인코딩
le_weekday = LabelEncoder()
traindf['요일_le'] = le_weekday.fit_transform(traindf['요일'])

# (선택) 원본 object 컬럼을 대체하고 싶다면
traindf['역명'] = traindf['역명_le']
traindf['요일'] = traindf['요일_le']
traindf.drop(columns=['역명_le','요일_le'], inplace=True)

In [299]:
traindf

,역명,이용량,월,일,년,요일,PM10수치,PM2.5수치,호선,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 풍속(m/s),최대 순간 풍속 풍향(deg)
0,0,4298.0,1,1,2023,4,26.0,14.0,1,2.3,-2.3,7.1,0.0,9.5,1.1,310.8
1,1,5624.0,1,1,2023,4,41.0,34.0,2,1.5,-4.1,7.3,0.0,10.2,2.2,17.7
2,2,4337.0,1,1,2023,4,40.0,31.0,2,1.5,-4.1,7.3,0.0,10.2,2.2,17.7
3,3,1550.0,1,1,2023,4,45.0,34.0,3,2.3,-2.3,7.1,0.0,9.5,1.1,310.8
4,4,5150.0,1,1,2023,4,42.0,31.0,2,2.3,-2.3,7.1,0.0,9.5,1.1,310.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66516,86,710.0,12,31,2024,6,39.0,17.0,3,3.1,-3.5,7.7,0.0,12.7,1.9,274.7
66517,87,6099.0,12,31,2024,6,26.0,14.0,1,4.2,0.7,7.2,0.0,10.8,1.4,293.0
66518,88,7165.0,12,31,2024,6,38.0,14.0,1,3.8,-0.4,7.1,0.0,9.9,1.7,237.9
66519,89,9447.0,12,31,2024,6,30.0,13.0,1,2.2,-5.3,7.2,0.0,12.9,2.6,323.1


In [300]:
# 예: '이용량'을 예측한다고 가정
X = traindf.drop(columns=['이용량'])
y = traindf['이용량']

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.3,      # 검증셋 비율(예: 20%)
    random_state=42     # 재현성 확보
)

## LIGHTGBM

In [301]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

model = lgb.LGBMRegressor(n_estimators=1000, 
                          learning_rate=0.01, 
                          max_depth=8,
                          random_state=42) 
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
print("Val RMSE:", np.sqrt(mean_squared_error(y_val, y_pred)))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001021 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1733
[LightGBM] [Info] Number of data points in the train set: 46564, number of used features: 15
[LightGBM] [Info] Start training from score 8573.977185
Val RMSE: 1346.8977451774347


In [302]:
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)


# 1) RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
# 2) MAE
mae  = mean_absolute_error(y_val, y_pred)
# 3) MAPE
mape = mean_absolute_percentage_error(y_val, y_pred)
# 4) R²
r2   = r2_score(y_val, y_pred)

print(f"Val RMSE : {rmse:.3f}")
print(f"Val MAE  : {mae:.3f}")
print(f"Val MAPE : {mape:.2%}")
print(f"Val R²   : {r2:.4f}")

Val RMSE : 1346.898
Val MAE  : 821.736
Val MAPE : 14.39%
Val R²   : 0.9602


# TEST예측

In [303]:
# 1) testdf에도 train에서 쓰던 전처리 동일하게 적용
testdf.drop(columns=['일자','날짜'], inplace=True)
testdf['역명'] = le_station.transform(testdf['역명'])
testdf['요일'] = le_weekday.transform(testdf['요일'])

# 2) 피처 리스트 정의
FEATURES = [c for c in traindf.columns if c != '이용량']

# 3) 예측
X_test = testdf[FEATURES]
testdf['예측_이용량'] = model.predict(X_test)

In [304]:
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)

# 실제값·예측값
y_test = testdf['이용량']
y_pred = testdf['예측_이용량']

# 1) RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# 2) MAE
mae  = mean_absolute_error(y_test, y_pred)
# 3) MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)
# 4) R²
r2   = r2_score(y_test, y_pred)

print(f"Test RMSE : {rmse:.3f}")
print(f"Test MAE  : {mae:.3f}")
print(f"Test MAPE : {mape:.2%}")
print(f"Test R²   : {r2:.4f}")

Test RMSE : 2186.788
Test MAE  : 1225.777
Test MAPE : 24.12%
Test R²   : 0.9004


In [305]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66521 entries, 0 to 66520
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   역명                66521 non-null  int64  
 1   이용량               66521 non-null  float64
 2   월                 66521 non-null  int64  
 3   일                 66521 non-null  int64  
 4   년                 66521 non-null  int64  
 5   요일                66521 non-null  int64  
 6   PM10수치            66521 non-null  float64
 7   PM2.5수치           66521 non-null  float64
 8   호선                66521 non-null  int64  
 9   평균기온(°C)          66521 non-null  float64
 10  최저기온(°C)          66521 non-null  float64
 11  최고기온(°C)          66521 non-null  float64
 12  일강수량(mm)          66521 non-null  float64
 13  최대 순간 풍속(m/s)     66521 non-null  float64
 14  평균 풍속(m/s)        66521 non-null  float64
 15  최대 순간 풍속 풍향(deg)  66521 non-null  float64
dtypes: float64(10), int64(6)
memory usage: 8

## RANDOMFOREST

In [306]:
#test데이터 초기화
testdf.drop(columns='예측_이용량', inplace = True)

In [307]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.3,      # 검증셋 비율
    random_state=42     # 재현성 확보
)
# 1) 모델 정의
rf = RandomForestRegressor(
    n_estimators=1000,      # 나무 개수
    max_depth=None,        # 깊이 제한 없앰
    min_samples_leaf=5,    # 말단 노드 최소 샘플
    n_jobs=-1,             # 모든 CPU 코어 사용
    random_state=42
)

# 2) 훈련
rf.fit(X_train, y_train)

# 3) 검증 예측
y_val_pred = rf.predict(X_val)

# 4) 성능 평가
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
mae  = mean_absolute_error(y_val, y_val_pred)
mape = mean_absolute_percentage_error(y_val, y_val_pred)
r2   = r2_score(y_val, y_val_pred)

print(f"RF Val RMSE : {rmse:.3f}")
print(f"RF Val MAE  : {mae:.3f}")
print(f"RF Val MAPE : {mape:.2%}")
print(f"RF Val R²   : {r2:.4f}")

RF Val RMSE : 1275.307
RF Val MAE  : 604.681
RF Val MAPE : 9.94%
RF Val R²   : 0.9643


In [308]:
testdf

,역명,이용량,월,일,년,요일,PM10수치,PM2.5수치,호선,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 풍속(m/s),최대 순간 풍속 풍향(deg)
0,0,4330,1,1,2025,2,33.0,17.0,1,3.5,-1.7,9.3,0.0,4.4,0.7,149.8
1,1,5786,1,1,2025,2,35.0,18.0,2,2.9,-3.5,8.8,0.0,4.7,1.1,275.0
2,2,4683,1,1,2025,2,30.0,14.0,2,2.9,-3.5,8.8,0.0,4.7,1.1,275.0
3,3,1562,1,1,2025,2,33.0,17.0,3,3.5,-1.7,9.3,0.0,4.4,0.7,149.8
4,4,4113,1,1,2025,2,30.0,13.0,2,3.5,-1.7,9.3,0.0,4.4,0.7,149.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10915,86,1049,4,30,2025,2,40.0,19.0,3,18.4,8.5,29.3,0.0,7.9,1.2,226.4
10916,87,5213,4,30,2025,2,50.0,25.0,1,19.9,11.6,30.0,0.0,8.1,1.5,167.1
10917,88,7401,4,30,2025,2,36.0,18.0,1,18.9,9.2,28.5,0.0,8.2,1.7,243.4
10918,89,8745,4,30,2025,2,37.0,16.0,1,18.5,8.1,28.3,0.0,11.2,2.7,220.1


In [309]:
X_test = testdf.drop(columns='이용량')
y_test = testdf['이용량']

# 4) 예측 수행
testdf['예측_이용량_rf'] = rf.predict(X_test)

# 5) 테스트셋 성능 평가
rmse = np.sqrt(mean_squared_error(y_test, testdf['예측_이용량_rf']))
mae  = mean_absolute_error(y_test, testdf['예측_이용량_rf'])
mape = mean_absolute_percentage_error(y_test, testdf['예측_이용량_rf'])
r2   = r2_score(y_test, testdf['예측_이용량_rf'])

print(f"Test RMSE : {rmse:.3f}")
print(f"Test MAE  : {mae:.3f}")
print(f"Test MAPE : {mape:.2%}")
print(f"Test R²   : {r2:.4f}")

# 6) 결과 확인
testdf.head()

Test RMSE : 1803.732
Test MAE  : 889.351
Test MAPE : 16.05%
Test R²   : 0.9323


,역명,이용량,월,일,년,요일,PM10수치,PM2.5수치,호선,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 풍속(m/s),최대 순간 풍속 풍향(deg),예측_이용량_rf
0,0,4330,1,1,2025,2,33.0,17.0,1,3.5,-1.7,9.3,0.0,4.4,0.7,149.8,9038.381869
1,1,5786,1,1,2025,2,35.0,18.0,2,2.9,-3.5,8.8,0.0,4.7,1.1,275.0,12121.034429
2,2,4683,1,1,2025,2,30.0,14.0,2,2.9,-3.5,8.8,0.0,4.7,1.1,275.0,11018.120875
3,3,1562,1,1,2025,2,33.0,17.0,3,3.5,-1.7,9.3,0.0,4.4,0.7,149.8,2731.394109
4,4,4113,1,1,2025,2,30.0,13.0,2,3.5,-1.7,9.3,0.0,4.4,0.7,149.8,11218.194704


In [310]:
import pandas as pd
fi = pd.Series(rf.feature_importances_,index= FEATURES).sort_values(ascending=False)
print(fi)

역명                  0.670514
호선                  0.227161
요일                  0.059061
PM10수치              0.013949
최저기온(°C)            0.005265
월                   0.005098
PM2.5수치             0.003452
최고기온(°C)            0.002959
평균기온(°C)            0.002768
일                   0.002394
최대 순간 풍속 풍향(deg)    0.002173
최대 순간 풍속(m/s)       0.001767
일강수량(mm)            0.001560
평균 풍속(m/s)          0.001046
년                   0.000835
dtype: float64
